In [2]:
import multiprocessing
import time

# Função global que será usada pelos processos de CPU
def cpu_task():
    while True:
        _ = 3.1415 * 2.7182  # Operação simples para manter a CPU ocupada

def stress_cpu():
    print("Estressando a CPU...")
    
    processes = []
    for _ in range(multiprocessing.cpu_count()):
        p = multiprocessing.Process(target=cpu_task)
        p.start()
        processes.append(p)

    time.sleep(10)  # Mantém o teste por 10 segundos
    for p in processes:
        p.terminate()
    print("Teste de CPU finalizado!")

# Executar o teste de CPU
stress_cpu()

Estressando a CPU...
Teste de CPU finalizado!


In [3]:
#Teste Memoria Ram
import numpy as np

def stress_ram():
    print("Estressando a memória RAM...")
    arrays = []
    try:
        while True:
            arrays.append(np.random.random((100, 100)))  # Aloca arrays de 1.000 x 1.000 até a memória acabar
    except MemoryError:
        print("Memória RAM esgotada!")
    print("Teste de RAM finalizado!")

# Executar o teste de RAM
stress_ram()


Estressando a memória RAM...
Memória RAM esgotada!
Teste de RAM finalizado!


In [15]:
import pyopencl as cl
import numpy as np
import time

def stress_gpu_opencl():
    print("Estressando a GPU (usando OpenCL)...")

    # Configurar o contexto e a fila de comando
    platforms = cl.get_platforms()
    devices = platforms[0].get_devices(cl.device_type.GPU)
    context = cl.Context([devices[0]])
    queue = cl.CommandQueue(context)

    # Kernel simples para estressar a GPU
    kernel_code = """
    __kernel void stress_gpu(__global float* a) {
        int idx = get_global_id(0);
        a[idx] = a[idx] * a[idx];
    }
    """

    # Compilar o kernel
    program = cl.Program(context, kernel_code).build()

    # Dados para o kernel
    a = np.random.randn(1000000).astype(np.float32)  # Tamanho ajustado para 1 milhão de elementos
    a_gpu = cl.Buffer(context, cl.mem_flags.READ_WRITE | cl.mem_flags.COPY_HOST_PTR, hostbuf=a)

    # Executar o kernel várias vezes para estressar a GPU
    for _ in range(100):
        program.stress_gpu(queue, (len(a),), None, a_gpu)

    # Sincronizar a execução
    queue.finish()
    print("Teste de GPU (OpenCL) finalizado!")

# Executar o teste de GPU com OpenCL
try:
    stress_gpu_opencl()
except cl.RuntimeError as e:
    print(f"Erro no OpenCL: {e}")
except cl.LogicError as e:
    print(f"Erro lógico no OpenCL: {e}")


Estressando a GPU (usando OpenCL)...
Erro lógico no OpenCL: clGetPlatformIDs failed: PLATFORM_NOT_FOUND_KHR


In [5]:
#Testes Disco
import os

def stress_disk():
    print("Estressando o disco (HD)...")
    file_path = "teste_hd.tmp"
    try:
        with open(file_path, "wb") as f:
            for _ in range(1000):
                f.write(os.urandom(1024 * 1024))  # Grava 1MB de dados aleatórios 1000 vezes
        print("Teste de escrita finalizado!")

        with open(file_path, "rb") as f:
            while f.read(1024 * 1024):
                pass  # Leitura do arquivo criado
        print("Teste de leitura finalizado!")
    finally:
        if os.path.exists(file_path):
            os.remove(file_path)  # Remove o arquivo temporário
    print("Teste de disco finalizado!")

# Executar o teste de disco
stress_disk()


Estressando o disco (HD)...
Teste de escrita finalizado!
Teste de leitura finalizado!
Teste de disco finalizado!
